In [2]:
from __future__ import absolute_import, division, print_function

In [1]:
# Data Augmentation 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size=16

train_aug = ImageDataGenerator(rescale=1.0/255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_aug = ImageDataGenerator(rescale=1.0/255)

train_generator = train_aug.flow_from_directory('GoT-images/train',
                                                target_size=(224,224),
                                                batch_size=batch_size,
                                                shuffle=True,
                                                class_mode='categorical')

validation_generator = test_aug.flow_from_directory('GoT-images/valid',
                                                   target_size=(224,224),
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   class_mode='categorical')


Using TensorFlow backend.


Found 37661 images belonging to 2 classes.
Found 7617 images belonging to 2 classes.


In [2]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization 
from keras import regularizers

# Version 2.0
# - removed max pooling layers
# - added additional conv layers with stride 2
# - increased number of filters (64,128,256,512,1024)
# - 2 fully connected layers w/ 1000 neurons 

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(filters=64, kernel_size=3, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))


model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(filters=128, kernel_size=3, strides=2, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))


model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(filters=256, kernel_size=3, strides=2, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))


model.add(Conv2D(filters=512, kernel_size=3, strides=1, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(filters=512, kernel_size=3, strides=2, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))


model.add(Conv2D(filters=512, kernel_size=3, strides=1, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(filters=512, kernel_size=3, strides=2, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(filters=1024, kernel_size=3, strides=2, padding='same')) 
model.add(BatchNormalization())
model.add(Activation('relu'))



model.add(Flatten())

model.add(Dense(1000))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7)) 

model.add(Dense(1000))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5)) 

model.add(Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
activation_2 (Activation)    (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

In [3]:
from keras.optimizers import Adam

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [7]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Training the model

epochs = 50

nb_training_size = 37661.0
nb_validation_size = 7617.0

checkpointer = ModelCheckpoint(filepath='saved_models/ver2_6_weights.hdf5', 
                               verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss', patience=5)


model.fit_generator(train_generator,
                   steps_per_epoch=nb_training_size//batch_size,
                   epochs=epochs,
                   validation_data=validation_generator,
                   validation_steps=nb_validation_size//batch_size,
                   verbose=1,
                   callbacks=[checkpointer])

Epoch 1/50
1176/1176 [==============================] - 1555s - loss: 0.7780 - acc: 0.5330 - val_loss: 0.7520 - val_acc: 0.5619
Epoch 2/50
1176/1176 [==============================] - 1549s - loss: 0.6812 - acc: 0.6075 - val_loss: 0.6037 - val_acc: 0.6708
Epoch 3/50
1176/1176 [==============================] - 1587s - loss: 0.6310 - acc: 0.6577 - val_loss: 0.5947 - val_acc: 0.6730
Epoch 4/50
1176/1176 [==============================] - 1554s - loss: 0.6101 - acc: 0.6816 - val_loss: 0.5474 - val_acc: 0.7209
Epoch 5/50
1176/1176 [==============================] - 1557s - loss: 0.5826 - acc: 0.6997 - val_loss: 0.5501 - val_acc: 0.7275
Epoch 6/50
1176/1176 [==============================] - 1563s - loss: 0.5676 - acc: 0.7136 - val_loss: 0.5720 - val_acc: 0.7197
Epoch 7/50
1176/1176 [==============================] - 1558s - loss: 0.5454 - acc: 0.7303 - val_loss: 0.5038 - val_acc: 0.7494
Epoch 8/50
1176/1176 [==============================] - 1557s - loss: 0.5276 - acc: 0.7415 - val_loss: 0

KeyboardInterrupt: 

In [8]:
#val_loss 0.33979 val_acc 0.8451
#training_loss 0.3303 training_accuracy 0.8527
model.save_weights('saved_models/ver_2_6_weights_resume.hdf5')

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Training resumed 

epochs = 25

nb_training_size = 37661.0
nb_validation_size = 7617.0

checkpointer = ModelCheckpoint(filepath='saved_models/ver2_6_weights_09142017.hdf5', 
                               verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss', patience=5)


model.fit_generator(train_generator,
                   steps_per_epoch=nb_training_size//batch_size,
                   epochs=epochs,
                   validation_data=validation_generator,
                   validation_steps=nb_validation_size//batch_size,
                   verbose=1,
                   callbacks=[checkpointer])

Epoch 1/25
1176/1176 [==============================] - 1582s - loss: 0.3244 - acc: 0.8571 - val_loss: 0.3536 - val_acc: 0.8376
Epoch 2/25
1176/1176 [==============================] - 1562s - loss: 0.3115 - acc: 0.8612 - val_loss: 0.3787 - val_acc: 0.8327
Epoch 3/25
1176/1176 [==============================] - 1563s - loss: 0.3034 - acc: 0.8674 - val_loss: 0.3486 - val_acc: 0.8450
Epoch 4/25
1176/1176 [==============================] - 1562s - loss: 0.2997 - acc: 0.8673 - val_loss: 0.3897 - val_acc: 0.8276
Epoch 5/25
1176/1176 [==============================] - 1559s - loss: 0.2880 - acc: 0.8741 - val_loss: 0.3593 - val_acc: 0.8443
Epoch 6/25
1176/1176 [==============================] - 1562s - loss: 0.2828 - acc: 0.8742 - val_loss: 0.3921 - val_acc: 0.8323
Epoch 7/25
1176/1176 [==============================] - 1557s - loss: 0.2773 - acc: 0.8813 - val_loss: 0.6165 - val_acc: 0.7872
Epoch 8/25
1176/1176 [==============================] - 1559s - loss: 0.2616 - acc: 0.8874 - val_loss: 0

KeyboardInterrupt: 

In [6]:
#val_loss 0.3486 val_acc 0.8450
#training_loss 0.1431 training_accuracy 0.9436
model.save_weights('saved_models/ver2.0_weights_final.hdf5')